# Profile wavebank indexing
This notebook is intended to help profile the 

In [ ]:
from pathlib import Path

import obspy

import obsplus

In [ ]:
test_dir = Path("archive")
index_file = test_dir / ".index.h5"

if index_file.exists():
    index_file.unlink()

In [ ]:
# make test directory
if not test_dir.exists():
    test_dir.mkdir(exist_ok=True, parents=True)
    endtime = 0
    for a in range(4000):
        st = obspy.read()
        for tr in st:
            tr.stats.starttime = obspy.UTCDateTime(endtime)
        st.write(str(test_dir / f"{a}.mseed"))
        endtime = tr.stats.endtime

In [ ]:
wb = obsplus.WaveBank(test_dir)


def index_update():
    if index_file.exists():
        index_file.unlink()
    wb.update_index()

In [ ]:
# %timeit index_update()

## Profile yielding traces

In [ ]:
index = wb.read_index()
start = index.starttime.min()
end = start + 48 * 3600 * 2

In [ ]:
%%time
for st in wb.yield_waveforms(starttime=start, endtime=end, duration=3600, overlap=2):
    pass

## Profile getting stream from bank

In [ ]:
# profile loading and joining many traces
%timeit st = wb.get_waveforms(starttime=start, endtime=start+3600*2)

In [ ]:
# profile trimming part of a trace
%timeit st = wb.get_waveforms(starttime=start + 10, endtime=start + 70)

In [ ]:
# %load_ext snakeviz

In [ ]:
# %%snakeviz
# st = wb.get_waveforms(starttime=start, endtime=start+3600*2)